In [35]:
# Flatten Harvard data

# For data manipulation
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json
import math

In [4]:
playerPath = os.path.join(PROJ_ROOT, "data", "interim", "harvard_players.json")

In [5]:
players = pd.read_json(playerPath)

In [6]:
players.head()

,B/T,City,Event,High School,Hometown,Ht.,Name,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport
0,R/R,Long Beach,None,Woodrow Wilson,"Long Beach, Calif.",6-2,Tyler Albright,/sports/bsb/2008-09/bios/albright_tyler,6,C,Calif.,None,None,190,Jr.,2008-09,bsb
1,R/R,Austin,None,St. Stephens Episcopal,"Austin, Texas",5-10,Cole Arledge,/sports/bsb/2008-09/bios/arledge_cole,8,C,Texas,None,None,180,Jr.,2008-09,bsb
10,S/R,Winnetka,None,New Trier,"Winnetka, Ill.",6-3,Zach Hofeld,/sports/bsb/2008-09/bios/hofeld_zach,29,RHP,Ill.,None,None,205,Jr.,2008-09,bsb
100,L/R,Charlottesville,None,St. Anne's-Belfield,"Charlottesville, Va.",5-11,Jake Allen,/sports/bsb/2016-17/bios/allen_jake_gyf5,10,C,Va.,None,None,185,So.,2016-17,bsb
1000,None,Greendale,None,Greendale HS,"Greendale, Wis.",6-8,Eric Wanta,/sports/mbkb/1984-85/Bios/Eric_Wanta,15,Forward,Wis.,None,None,210,Freshman,1984-85,mbkb


In [7]:
players.shape

(10896, 17)

In [8]:
bio_parsed_path = os.path.join(PROJ_ROOT, "data", "interim", "harvard_bio_parsed.csv.gz")
bios = pd.read_csv(bio_parsed_path, compression="gzip")


In [15]:
bios.shape
bios.columns = ["Name_link", "synopsis", "headers"]

In [18]:
combined = players.merge(bios, on="Name_link", how="inner")

In [19]:
combined.shape

(10895, 19)

In [43]:
combined.head()

,B/T,City,Event,High School,Hometown,Ht.,Name,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport,synopsis,headers
0,R/R,Long Beach,None,Woodrow Wilson,"Long Beach, Calif.",6-2,Tyler Albright,/sports/bsb/2008-09/bios/albright_tyler,6,C,Calif.,None,None,190,Jr.,2008-09,bsb,\nJunior (2010):\n\n Named team's captain prio...,"{u'Position:': u'C', u'Year:': u'Jr.', u'Heigh..."
1,R/R,Long Beach,None,Woodrow Wilson,"Long Beach, Calif.",6-2,Tyler Albright,/sports/bsb/2008-09/bios/albright_tyler,6,C,Calif.,None,None,190,Jr.,2009-10,bsb,\nJunior (2010):\n\n Named team's captain prio...,"{u'Position:': u'C', u'Year:': u'Jr.', u'Heigh..."
2,R/R,Austin,None,St. Stephens Episcopal,"Austin, Texas",5-10,Cole Arledge,/sports/bsb/2008-09/bios/arledge_cole,8,C,Texas,None,None,180,Jr.,2008-09,bsb,\nJunior (2010):\n\n Started at catcher agains...,"{u'Position:': u'C', u'Year:': u'Jr.', u'Heigh..."
3,R/R,Austin,None,St. Stephens Episcopal,"Austin, Texas",5-10,Cole Arledge,/sports/bsb/2008-09/bios/arledge_cole,8,C,Texas,None,None,180,Jr.,2009-10,bsb,\nJunior (2010):\n\n Started at catcher agains...,"{u'Position:': u'C', u'Year:': u'Jr.', u'Heigh..."
4,S/R,Winnetka,None,New Trier,"Winnetka, Ill.",6-3,Zach Hofeld,/sports/bsb/2008-09/bios/hofeld_zach,29,RHP,Ill.,None,None,205,Jr.,2008-09,bsb,\nSophomore (2009):\n\nStruck out three batter...,"{u'Position:': u'RHP', u'Year:': u'Jr.', u'Hei..."


In [39]:
def formatBio(row):
    synopsis = row["synopsis"]
    if type(synopsis) == float:
        return ""
    return re.sub("\n", "", synopsis)

In [44]:
combined["Bio"] = combined.apply(formatBio, axis=1)

In [47]:
del(combined["synopsis"])

In [49]:
playersSorted = combined.sort_values(["season"], ascending=False)

In [50]:
groupedFirst = playersSorted.groupby(by=["Name"]).first()
groupedLast = playersSorted.groupby(by=["Name"]).last()

In [60]:
len(groupedLast)

4270

In [ ]:
for i in range(len(groupedLast)):
    first = groupedFirst.iloc[i]
    groupedLast.iloc[i]["season"] = groupedLast.iloc[i]["season"][:-2] + first["season"][-2:]